In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from tensorflow import keras as k

import tensorflow as tf
import pandas as pd
import numpy as np

# %pip install d2l==1.0.0a1.post0
from d2l import tensorflow as d2l


In [6]:
data = pd.read_csv('./input/dataset.csv')
# dataImputed = pd.read_csv('./input/datasetImputed.csv')

In [7]:
# è stata riscritta una funzione di reshape inutile 
# e.g. (a_prev=x_train, season=trn_ssn)
def reshape_to_inputshape(a_prev,season):
    totalMatches = len(season)*38
    input_step = int(a_prev.shape[0]/totalMatches)
    prev_f = a_prev.shape[1]
    return np.reshape(a_prev, (totalMatches, input_step, prev_f))

In [8]:
features = ['HomeTeam', 'AwayTeam', 
            'HTeamEloScore', 'ATeamEloScore', 
            'HTdaysSinceLastMatch', 'ATdaysSinceLastMatch', 
            'HTW_rate', 'ATW_rate', 'ATD_rate', 'HTD_rate', 
            '7_HTW_rate', '12_HTW_rate', '7_ATW_rate', '12_ATW_rate', 
            '7_HTD_rate', '12_HTD_rate', '7_ATD_rate', '12_ATD_rate',
            '7_HTL_rate', '12_HTL_rate', '7_ATL_rate', '12_ATL_rate',
            '5_HTHW_rate', '5_ATAW_rate']

X = pd.get_dummies(data[features])

# Se non cambiamo nulla il OneHotEncoder assegna:
# A -> 1 0 0
# D -> 0 1 0
# H -> 0 0 1
y = data[['FTR']].to_numpy().ravel().reshape(-1, 1)
y = OneHotEncoder(sparse=False).fit_transform(y)
X_imputed = SimpleImputer().fit_transform(X)



In [9]:
# è stata riscritta una funzione di reshape inutile 
# e.g. (a_prev=x_train, season=trn_ssn)
def reshape_to_inputshape(a_prev,season):
    totalMatches = len(season)*38
    input_step = int(a_prev.shape[0]/totalMatches)
    prev_f = a_prev.shape[1]
    return np.reshape(a_prev, (totalMatches, input_step, prev_f))

In [10]:
trn_ssn = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
trn_ssn_len = len(trn_ssn)
tst_ssn = [2016,2017,2018] 
tst_ssn_len = len(tst_ssn)



test_size = float(tst_ssn_len)/(tst_ssn_len+trn_ssn_len)
#Split X and Y into training and Test Sets
x_train, x_test, y_train, y_test = train_test_split(data, y, shuffle=False, test_size=test_size)

#Setup XY to have 10 game steps
x_train = reshape_to_inputshape(x_train,trn_ssn)

y_train = reshape_to_inputshape(y_train,trn_ssn)
y_train = np.moveaxis(y_train, 0, 1)
x_test = reshape_to_inputshape(x_test,tst_ssn)
y_test = reshape_to_inputshape(y_test,tst_ssn)
y_test = np.moveaxis(y_test, 0, 1)

Tx= x_train.shape[1] #Time steps
Ty= y_train.shape[0] #Time Steps
num_features = x_train.shape[2] #Features per step

inputs = tf.keras.Input(shape=(Tx, num_features))

ValueError: Must pass 2-d input. shape=(304, 10, 118)

In [ ]:
lst = tf.keras.models.Sequential([
    # tf.keras.Input(shape=(Tx, num_features)),
    # k.layers.Lambda(lambda z: inputs[:, 1, :]),
    k.layers.Flatten(),
    k.layers.Dense(4000),
    k.layers.Dense(240),
    k.layers.Dense(3, activation='softmax')
])

In [ ]:
lst.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001)
#    ,metrics=[tf.keras.metrics.Accuracy()]
)
lst(x_train)
lst.summary()

# Preprocessamento dati

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from IPython.display import display

#Create X and Y
X = pd.get_dummies(data[['HomeTeam', 'AwayTeam', 'HTeamEloScore', 'ATeamEloScore', 'HTdaysSinceLastMatch',
                            'ATdaysSinceLastMatch', 'HTW_rate', 'ATW_rate',
                            'ATD_rate', 'HTD_rate', 
                '7_HTW_rate', '12_HTW_rate', '7_ATW_rate', '12_ATW_rate', 
                '7_HTD_rate', '12_HTD_rate', '7_ATD_rate', '12_ATD_rate',
                '7_HTL_rate', '12_HTL_rate', '7_ATL_rate', '12_ATL_rate',
                '5_HTHW_rate', '5_ATAW_rate']])
Y = data[['FTR']]

#X preprocessing
imputer = SimpleImputer()
X_imputed = imputer.fit_transform(X)

#Split X and Y into training and Test Sets
x_train, x_test, y_train, y_test = train_test_split(X_imputed, Y, shuffle=True)


# Logistic Regression

In [ ]:
#Logistic Regression Model Setup
model = LogisticRegression()
model.fit(x_train, y_train)

#Logistic Regression Model Metrics
print("Logestic Regression")
print("Train Score: ", model.score(x_train, y_train))
print("Test Score: ", model.score(x_test, y_test))
print(classification_report(y_test, model.predict(x_test), digits=3))

# Random forest

In [ ]:
#Forest model setup
forest = RandomForestClassifier(n_estimators=2, random_state=2)
forest.fit(x_train, y_train)

#Forest Model Metrics
print("Forest Classifier")
print("Train Score: ", forest.score(x_train, y_train))
print("Test Score: ", forest.score(x_test, y_test))
print(classification_report(y_test, forest.predict(x_test), digits=3))

print(y_test.shape)
print(forest.predict(x_test).shape)

# Neural network

In [ ]:
import tensorflow.compat.v1 as tf

#make TF work like the TF in the HWs
tf.disable_v2_behavior()

Y = data[['ordinalHR']]
print(Y)
Y = data[['ordinalHR']].to_numpy().ravel()*2
print(Y)

#Neural Network Setup
n_inputs = X.shape[1]
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 3

#Tensorflow X and Y
X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = 'X')
Y = tf.placeholder(tf.int32, shape = (None), name = 'Y')

#Function used to better display model metrics
def convert_ordinalHR(x):
    y=[]
    for i in range(x.size):
        if x[i] == 0:
            y.append('A')
        elif x[i] == 2:
            y.append('H')
        elif x[i] == 1:
            y.append('D')
    return y

#General Function for neural layer setup
def neuron_layer(X, n_neurons, name, activation = None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name = 'kernel')
        b = tf.Variable(tf.zeros([n_neurons]), name='bias')
        L2 = tf.nn.l2_loss(W)
        Z = tf.matmul(X,W)+b
        if activation is not None:
            return activation(Z), L2
        else:
            return Z, L2

#Tensorflow neural layer setup
with tf.name_scope('dnn'):
    hidden1, L2_1 = neuron_layer(X, n_hidden1, name='hidden1', activation=tf.nn.relu)
    hidden2, L2_2 = neuron_layer(hidden1, n_hidden2, name='hidden2', activation=tf.nn.relu)
    logits, L2_3 = neuron_layer(hidden2, n_outputs, name='outputs', activation=None)

#Loss function
beta=0.01
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = Y, logits=logits)
    loss = tf.reduce_mean(xentropy+beta*(L2_1+L2_2+L2_3), name='loss')
    
#Optimizer
learning_rate = 0.000001
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

#Metric analysis setup
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, Y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    preds = tf.argmax(input=logits, axis=1)
    
#Initialize the above tensorflow variables    
init = tf.global_variables_initializer()

#Runtime and batchsetup
n_epochs = 30000
batch_size = 50

#Run the model
with tf.Session() as sess:
    init.run()
    #Model Loop
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: x_train, Y: y_train})
        acc_train = accuracy.eval(feed_dict={X: x_train, Y: y_train})
        acc_val = accuracy.eval(feed_dict={X: x_test, Y: y_test})
        if(epoch % 200 == 0 or epoch==n_epochs-1):
            print(epoch, 'Train accuracy:', acc_train, 'Val accuracy:', acc_val)
    
    #Retrive Metrics
    acc_val = accuracy.eval(feed_dict={X: x_test, Y: y_test})
    preds = preds.eval(feed_dict = {X:x_test})
    log = logits.eval(feed_dict = {X:x_test})
    preds=convert_ordinalHR(preds)
    y_test=convert_ordinalHR(y_test)
    
    #Print Metrics
    print('Train accuracy:', acc_train, 'Val accuracy:', acc_val)
    print(classification_report(y_test, preds, digits=3))

# Provare altre architetture deep, magari altre RNN (e.g., GRU, bidirectional, deep RNN, ...) e vedere se si comportano meglio o peggio della LSTM

In [ ]:
n = 10
m = 5

for (x,y) in [(i,j) for i in range(n) for j in range(m)]:
    print(f"(",x,y,")")
    

In [ ]:
inputs = k.layers.Input(shape=(10,94))
lstm1 = k.layers.LSTM(100, return_sequences=True)(inputs)
lstm2 = k.layers.LSTM(50, return_sequences=True)(inputs)
concateneted = k.layers.Concatenate()([
    lstm1,
    lstm2
])

out = k.layers.Dropout(0.5)(concateneted)
out = k.layers.Dense(1000, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(3, activation="softmax")(out)

model = k.models.Model(inputs=inputs, outputs=out)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.fit(x_prova, y_prova, epochs=500, callbacks=[callback])

In [ ]:
model.evaluate(x_prova_test, y_prova_test)

In [ ]:
def revert_yoh(Y):
    Y_new = np.empty([Y.shape[0],Y.shape[1]], dtype="<U1")
    #Y_new = np.zeros((Y.shape[0],Y.shape[1]))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if (Y[i, j] == 0):
                Y_new[i, j]= 'A'
            elif (Y[i, j] == 1):
                Y_new[i, j]= 'D'
            elif (Y[i, j] == 2):
                Y_new[i, j]='H'
    return Y_new

y_pred = model.predict(x_prova_test)
y_predm = np.asarray(y_pred)
y_predm = np.argmax(y_predm, axis=2)
y_testm = np.argmax(y_prova_test, axis=2)

y_pred_train = model.predict(x_prova)
y_pred_train = np.asarray(y_pred_train)
y_predm_train = np.argmax(y_pred_train, axis=2)
y_trainm = np.argmax(y_prova, axis = 2)

y_predm = revert_yoh(y_predm).ravel()
y_testm = revert_yoh(y_testm).ravel()

y_predm_train = revert_yoh(y_predm_train).ravel()
y_trainm = revert_yoh(y_trainm).ravel()

#Model Metrics
print(classification_report(y_testm, y_predm, digits=3))